In [2]:
from py2cytoscape import cyrest
from statistics import *
from math import *
from numpy.random import randint

In [29]:
#Get data from current network
cy = cyrest.cyclient()
data = {}

#Doing both
raw_data = cy.node.get_properties(network='CURRENT', nodeList='all', propertyList='Y Location, Height')
heights = []
for item in raw_data:
    for prop in item['visualProperties']:
        if prop['visualProperty'] == 'NODE_HEIGHT':
            heights.append(prop['value'])
min_height = min(heights)
magnitude = floor(log10(min_height / 100))
                  
if magnitude <= 0:
    for entry in raw_data:
        random = (randint(0, 10000) - 5000) * (10 ** (magnitude - 5))
        data[str(entry['SUID'])] = round(
            [
                prop['value'] 
                for prop in entry['visualProperties'] 
                if prop['visualProperty'] == 'NODE_Y_LOCATION'
            ][0],
            -magnitude
        ) #+ random
else:
    for entry in raw_data:
        random = (randint(0, 10000) - 5000) * (10 ** (magnitude - 5))
        data[str(entry['SUID'])] = round(
            [
                prop['value'] 
                for prop in entry['visualProperties'] 
                if prop['visualProperty'] == 'NODE_Y_LOCATION'
            ][0] 
            / 10 ** magnitude
        ) * (10 ** magnitude) #+ random
        
data

{'464': 309.0,
 '465': -325.1,
 '466': -322.7,
 '467': 323.4,
 '468': -187.5,
 '469': -552.9,
 '470': -310.6,
 '471': -156.2,
 '472': -562.5,
 '473': -427.3,
 '474': 69.2,
 '475': 171.9,
 '476': -45.6,
 '477': -451.4,
 '478': -57.7,
 '479': -572.1,
 '480': -574.6,
 '481': -444.2,
 '482': -427.3,
 '483': -437.0,
 '484': -463.5,
 '485': -303.4,
 '486': -593.8,
 '487': -449.0,
 '488': 152.7,
 '489': -38.4,
 '490': -548.1,
 '491': 71.6,
 '492': -165.8,
 '493': -305.8}

In [33]:
#Add variance of unclustered nodes as a cluster into variance within clusters
clusters = [{'members': [item[0]], 'values': [item[1]]} for item in data.items()]
iterations = 0

print(variance([cluster['values'][0] for cluster in clusters]))

not_clustered = data

while len(clusters) > 1:
    iterations += 1
    #Find closest clusters
    smallest_diff = float('Infinity')
    i_coord = None
    j_coord = None
    for i in range(len(clusters)-1):
        for j in range(i+1, len(clusters)):
            diff = abs(mean(clusters[i]['values']) - mean(clusters[j]['values']))
            if diff < smallest_diff:
                smallest_diff = diff
                i_coord = i
                j_coord = j
    #Merge closest clusters
    clusters[i_coord]['members'] += clusters[j_coord]['members']
    clusters[i_coord]['values'] += clusters[j_coord]['values']
    del(clusters[j_coord])
    
    sum_of_variances_within = 0
    #if len(not_clustered) > 0:
     #   for member in clusters[i_coord]['members']:
      #      if member in not_clustered:
       #         not_clustered.pop(member)
        #sum_of_variances_within += variance(not_clustered.values())
    
    not_clustered = []
    for cluster in clusters:
        if len(cluster['members']) > 1:
            sum_of_variances_within += variance(cluster['values'])/abs(mean(cluster['values']))
            #print(cluster['members'])
        else:
            not_clustered += cluster['values']
       
    #if len(not_clustered) > 1:
     #   sum_of_variances_within += variance(not_clustered)
        
    print(sum_of_variances_within)
    #print(len(clusters))

76448.32529885057
0.0
0.006397156819191349
0.047306247728281965
0.05619788094841189
0.06565290392937079
0.07110332688219193
0.09202975739990271
0.12641035085730049
0.1499998071572523
0.76714266430011
1.0533538444243349
1.1262014752697975
1.1923693532764899
1.5202630914169064
1.6395657875173864
3.036325541524847
3.293712479159038
4.429387155683993
4.575718660773561
4.75141161822544
5.979174922890224
6.225943809707684
29.781662004743197
69.09893232895128
76.96508623722372
91.3709454255104
134.07156590271057
854.3671289912328
300.7921307022594
